In [ ]:
# %%html
# <style>div.input {display:none;} div.output_stderr{display:none}</style>

[Signature Analysis Pipeline - Readme on github](https://github.com/KnowEnG/Signature_Analysis_Pipeline/blob/master/README.md)

# KnowEnG 's Signiture Analysis Pipeline notebook


In [1]:
import sys
sys.path.insert(1,'../notebooks_KnowEnG/src')
from Signature_Analysis_notebook_tools import show_signature_analysis_controls
show_signature_analysis_controls()

Box(children=(Dropdown(layout=Layout(width='50%'), options=('BENCHMARK_1_GS_cos.yml', 'BENCHMARK_1_GS_cos_copy.yml', 'BENCHMARK_1_GS_spearman.yml', 'View == Update List'), value='BENCHMARK_1_GS_cos.yml'), Button(description='View', style=ButtonStyle(), tooltip='visualize selected file')), layout=Layout(align_items='stretch', border='none', display='inline-flex', flex_flow='row', justify_content='space-between', width='100%'))

HTML(value='')

HBox(children=(Button(description='RUN', style=ButtonStyle(), tooltip='execute selected file'),), layout=Layout(display='flex', flex_flow='row', justify_content='flex-end', width='100%'))

HTML(value='')

Box(children=(Dropdown(layout=Layout(width='50%'), options=('result_similarity_spearman_Mon_04_Dec_2017_09_39_02.599036931_viz.tsv', 'View == Update List'), value='result_similarity_spearman_Mon_04_Dec_2017_09_39_02.599036931_viz.tsv'), Button(description='View', style=ButtonStyle(), tooltip='visualize selected file')), layout=Layout(align_items='stretch', border='none', display='inline-flex', flex_flow='row', justify_content='space-between', width='100%'))

HTML(value='')